In [1]:
# Importación de librerías
import pandas as pd  # type: ignore # Para manipulación de datos
import ast  # Para evaluar strings como expresiones de Python
import re  # Para expresiones regulares

In [2]:
# Carga de datos
# Cargar el archivo de business
business = pd.read_csv(r'business.csv')

# Cargar el archivo de categorías normalizadas
categorias = pd.read_csv(r'categorias_normalizadas.csv')

# Cargar el archivo de categorías normalizadas
ciudades = pd.read_csv(r'data_normalizada\ciudades_normal.csv')

# Cargar el archivo de categorías normalizadas
reviews = pd.read_csv(r'reviews_yelp.csv')

In [3]:
# Función para convertir strings a listas
def convertir_a_lista(valor):
    """
    Convierte un string que representa una lista en una lista real de Python.
    
    Parámetros:
    - valor: El valor a convertir, puede ser un string o una lista.
    
    Retorna:
    - Una lista si el valor era un string que representaba una lista.
    - Una lista con un solo elemento si el valor era un string que no representaba una lista.
    - Una lista vacía si el valor no era un string o si había un error en la conversión.
    """
    if isinstance(valor, str):
        try:
            lista = ast.literal_eval(valor)  # Convierte el string en una lista real
            if isinstance(lista, list):
                return lista
            return [lista]  # Si era un solo valor, convertirlo en lista
        except:
            return []  # Si hay un error, devolver una lista vacía
    return valor  # Si ya es una lista, dejarlo igual

# Aplicar la función a la columna 'categories' del DataFrame business
business['categories'] = business['categories'].dropna().apply(convertir_a_lista)

In [4]:
# Función para normalizar nombres de categorías
def normalizar_categoria(cat):
    """
    Normaliza el nombre de una categoría eliminando espacios extra, caracteres especiales, etc.
    
    Parámetros:
    - cat: El nombre de la categoría a normalizar.
    
    Retorna:
    - El nombre de la categoría normalizado o None si la categoría queda vacía.
    """
    cat = cat.lower().strip()  # Minúsculas y eliminar espacios extra
    cat = re.sub(r'\s*,\s*', ',', cat)  # Asegurar que las comas no tengan espacios extra
    cat = re.sub(r'\s*restaurant[s]?\b', '', cat)  # Eliminar 'restaurant' y variantes
    cat = re.sub(r'[^a-z\s,]', '', cat)  # Remover caracteres especiales excepto espacios y comas
    cat = re.sub(r'\s+', ' ', cat)  # Reemplazar múltiples espacios por uno solo
    cat = cat.strip()
    
    return cat if cat else None  # Devolver None si la categoría queda vacía

# Aplicar normalización a cada elemento dentro de las listas de categorías
business['categories'] = business['categories'].apply(
    lambda lista: list(set(filter(None, [normalizar_categoria(cat) for cat in lista]))) or ["restaurant"]
)

# %% Crear un diccionario de mapeo {Categoria: category_id}
category_map = dict(zip(categorias['category'], categorias['category_id']))

In [5]:
# Función para mapear categorías a IDs
def map_categories_to_ids(category_list):
    """
    Mapea una lista de categorías a sus respectivos IDs.
    
    Parámetros:
    - category_list: Lista de categorías a mapear.
    
    Retorna:
    - Lista de IDs correspondientes a las categorías.
    """
    return [category_map[c.strip().lower()] for c in category_list if c.strip().lower() in category_map]

# Aplicar la transformación
business['category_id'] = business['categories'].apply(map_categories_to_ids)

In [6]:
# Verificación de ciudades en business que no están en el DataFrame "ciudades"
# Asegurar que los nombres de ciudad en ambos DataFrames estén en el mismo formato
business['city_normalized'] = business['city'].str.strip().str.title()
ciudades['city_normalized'] = ciudades['city'].str.strip().str.title()

# Normalización de nombres de ciudades en `business` y `ciudades`
def normalizar_ciudad(nombre):
    """
    Normaliza el nombre de una ciudad eliminando abreviaciones, espacios extra, etc.
    
    Parámetros:
    - nombre: El nombre de la ciudad a normalizar.
    
    Retorna:
    - El nombre de la ciudad normalizado.
    """
    nombre = nombre.strip().title()  # Eliminar espacios y convertir en formato título
    nombre = nombre.replace("St.", "Saint")  # Reemplazar abreviaciones comunes
    nombre = nombre.replace("Ft.", "Fort")
    nombre = nombre.replace("Mt.", "Mount")
    nombre = nombre.replace(" N ", " North ").replace(" S ", " South ")
    nombre = nombre.replace(" E ", " East ").replace(" W ", " West ")
    nombre = nombre.replace("-", " ")  # Convertir guiones en espacios
    nombre = nombre.replace("'", "")  # Eliminar apóstrofes
    return nombre.strip()

# Aplicar normalización a ambas listas de ciudades
business['city_normalized'] = business['city'].apply(normalizar_ciudad)
ciudades['city_normalized'] = ciudades['city'].apply(normalizar_ciudad)

# Mapeo de nombres de ciudades
city_mapping_unificado = {
    "Islamorada, Village Of Islands": ["Islamorada"],
    "St. Cloud": ["St Cloud"],
    "Port St. Joe": ["Port St Joe"],
    "Wesley Chapel": ["Wesley Chapel South"],
    "Northdale": ["Greater Northdale"],
    "Carrollwood": ["Greater Carrollwood"],
    "Tampa": ["Ybor City"],
    "St. Pete Beach": ["St Pete Beach", "Pass-A-Grille Beach", "St. Pete Beach"],
    "Clearwater": ["Clearwater Beach"],
    "Rotonda": ["Rotonda West"],
    "Islamorada, Village Of Islands": ["Islamorada"],
    "Lake Worth Beach": ["Lake Worth"],
    "Plantation City": ["Plantation"],
    "Miami Gardens": ["Carol City"],
    "Hollywood": ["Hollywood Beach"],
    "Miami": ["Coconut Grove"],
    "St. James City": ["St James City"],
    "Port St. John": ["Port St John"],
    "Glen St. Mary": ["Glen St Mary"],
    "Laurel": ["Laurel Hill"],
    "Pensacola": ["Pensacola Beach", "Cantonment"],
    "Pace": ["Milton"],
    "Crestview": ["Holt", "Baker"],
    "Fort Walton Beach": ["Shalimar"],
    "Destin": ["Santa Rosa Beach", "Sandestin"],
    "Marianna": ["Alford"],
    "Panama City": ["Wewahitchka", "Ebro", "Southport"],
    "Panama City Beach": ["Inlet Beach", "Alys Beach", "Seacrest", "Rosemary Beach"],
    "DeFuniak Springs": ["Redbay", "Ponce De Leon"],
    "Brandon": ["Lithia"],
    "St. Pete Beach": ["Pass-A-Grille Beach", "St Pete Beach"],
    "Bradenton": ["Parrish"],
    "Sarasota": ["University Park", "Myakka City"],
    "Port Charlotte": ["Placida"],
    "Cape Coral": ["Boca Grande"],
    "Key West": ["Little Torch Key", "Summerland Key", "Naval Air Station Key West"],
    "Islamorada, Village Of Islands": ["Tavernier", "Islamorada"],
    "West Palm Beach": ["Singer Island", "Golden Lakes", "Loxahatchee"],
    "Boca Raton": ["Sandalfoot Cove"],
    "Naples": ["Everglades City", "Ochopee", "Goodland"],
    "Fort Myers": ["Miromar Lakes", "Tice"],
    "Sebring": ["Lorida", "Palmdale"],
    "Kissimmee": ["Reunion", "Intercession City"],
    "St. Cloud": ["Kenansville", "Harmony", "St Cloud"],
    "Lake Wales": ["River Ranch"],
    "Melbourne": ["Patrick AFB", "Melbourne Beach", "Patrick Space Force Base", "Patrick Afb"],
    "Jacksonville": ["St. Johns", "Ponte Vedra Beach", "Mayport", "St Johns"],
    "Fernandina Beach": ["Amelia Island"],
    "Palatka": ["Florahome", "Georgetown", "San Mateo", "East Palatka", "Satsuma"],
    "Gainesville": ["Keystone Heights", "Melrose", "Hawthorne", "Jonesville", "3720 Nw 13Th St Suite #14 Gainesville"],
    "Ocala": ["Fort Mccoy","Citra", "Sparr", "Fort McCoy", "Salt Springs", "Belleview", "Anthony"],
    "St. Augustine": ["Elkton", "Hastings", "St Augustine Beach"],
    "Sanford": ["Lake Monroe"],
    "Deltona": ["Osteen", "Pierson", "DeLand"],
    "Orlando": ["Chuluota", "Sand Lake"],
    "Mount Dora": ["Mt Dora", "Mt Plymouth"],
    "The Villages": ["Oxford", "Summerfield", "Weirsdale", "Sumterville"],
    "Beverly Hills": ["Pine Ridge"],
    "Lake City": ["Branford", "White Springs", "Wellborn", "Sanderson"],
    "Chiefland": ["Old Town"],
    "Perry": ["Steinhatchee", "Horseshoe Beach"],
    "Apalachicola": ["St George Island"],
    "Tallahassee": ["Lamont", "St Marks", "Wakulla Springs"],
    "Madison": ["Lee"],
    "Live Oak": ["Mayo", "Jennings"],
    "Lake Butler City": ["Lake Butler"],
    "Starke": ["Raiford"],
    "Brooksville" : ["Brooksville","Springhill"],
    "Holmes Beach" : ["Holmes Beach", "Westville"]
}

# Aplicar el diccionario de mapeo para corregir los nombres de las ciudades
business['city'] = business['city'].replace(city_mapping_unificado)

# Crear un diccionario inverso para el mapeo de ciudades
reverse_city_mapping = {alt: official for official, alternatives in city_mapping_unificado.items() for alt in alternatives}

# Aplicar el mapeo en la columna 'city' de business
business['city'] = business['city'].apply(lambda x: reverse_city_mapping.get(x.strip().title(), x.strip().title()))

# Volver a verificar cuántas ciudades coinciden con las oficiales
business['city_normalized'] = business['city'].str.strip().str.title()
ciudades['city_normalized'] = ciudades['city'].str.strip().str.title()

In [7]:
# Fusionar business con ciudades para obtener city_id
business = business.merge(
    ciudades[['city_normalized', 'city_id']],  # Solo tomamos city_id de ciudades
    left_on='city_normalized', 
    right_on='city_normalized', 
    how='left'  # Mantenemos todas las filas de business, aunque no haya coincidencia
)


# Verificar si quedaron ciudades sin city_id asignado
ciudades_sin_id = business[business['city_id'].isna()]

# Mostrar cuántas ciudades no obtuvieron un city_id
print(f"Ciudades sin city_id asignado: {len(ciudades_sin_id)}")

# Mostrar algunas ciudades sin city_id para depuración
print(ciudades_sin_id[['city', 'city_normalized']].drop_duplicates().head(20))


Ciudades sin city_id asignado: 0
Empty DataFrame
Columns: [city, city_normalized]
Index: []


In [8]:
# Unir los DataFrames en base a 'business_id'
reviews = reviews.merge(business[['business_id', 'id']], on='business_id', how='left')

# Eliminar la columna 'business_id'
reviews.drop(columns=['business_id'], inplace=True)

# Reordenar las columnas para que 'id' sea la primera
columnas_ordenadas = ['id'] + [col for col in reviews.columns if col != 'id']
reviews = reviews[columnas_ordenadas]
reviews = reviews.dropna(subset=['id'])

In [9]:
business_normal = business.drop(columns=['city', 'state','categories', 'city_normalized'])

In [10]:
# Definir el orden lógico de las columnas
column_order = [
    # Identificación
    'id', "business_id", "name",

    # Ubicación
    "address", "postal_code", "latitude", "longitude", 'city_id', 

    # Información general
    "stars", "review_count", "is_open", 'category_id',

    # Opciones de servicio
    "delivery", "takeout", "outdoor_seating", "drivethrough",

    # Características y comodidades
    "wheelchair_friendly", "alcohol_beverage", "dancing",
    "catering", "counter_service", "seating", "dogs_allowed",
    "bike_parking", "parking",

    # Horarios de comida
    "breakfast", "lunch", "dinner", "dessert",

    # Ambiente
    "casual", "romantic", "formal", "trendy",

    # Reservas y tiempos de espera
    "with_reservation", 

    # Entretenimiento y características adicionales
    "live_entertainment",

    # Público objetivo
    "groups", "kids_friendly",

    # Conectividad y tecnología
    "wifi", "bar_onsite",

    # Opciones de pago
    "credit_cards",
]

# Aplicar el orden de columnas asegurando que no haya errores por columnas faltantes
business_normal = business_normal[column_order]

# Guardar el DataFrame final a CSV
business_normal.to_csv(r'business_normal.csv', index=False)

In [11]:
def unificar_registros(df):
    """
    Unifica los registros duplicados en base a la columna 'id'.
    
    - Mantiene un valor arbitrario para 'gmap_id'.
    - Conserva el primer valor de 'name', 'street_address', 'postal_code', 'latitude', 'longitude', 'city_id'.
    - Une los valores únicos de 'category_id'.
    - Calcula el promedio ponderado de 'stars' usando 'review_count'.
    - Suma los valores de 'review_count'.
    - Para las columnas binarias, si hay al menos un 1, deja 1; si todos son 0, deja 0.
    
    Parámetros:
    - df: DataFrame con registros duplicados a consolidar.

    Retorna:
    - DataFrame sin duplicados con valores agregados correctamente.
    """

    # Columnas a mantener el primer valor (todas son iguales)
    keep_first = ['business_id', 'name', 'postal_code', 'latitude', 'longitude', 'city_id']

    # Reglas de agregación personalizadas
    agg_rules = {
        'category_id': lambda x: list(set().union(*x)) if x.dtype == 'O' else x,  # Unir listas y dejar valores únicos
        'stars': lambda x: (x * df.loc[x.index, 'review_count']).sum() / df.loc[x.index, 'review_count'].sum() if df.loc[x.index, 'review_count'].sum() > 0 else x.mean(),  # Promedio ponderado
        'review_count': 'sum'  # Sumar reviews
    }

    # Identificar las columnas binarias y aplicar "máximo" (si hay un 1, queda 1)
    binary_columns = [col for col in df.columns if col not in ['id'] + keep_first + list(agg_rules.keys())]
    for col in binary_columns:
        agg_rules[col] = 'max'

    # Aplicar reglas de agregación
    df_unificado = df.groupby('id', as_index=False).agg({**{col: 'first' for col in keep_first}, **agg_rules})

    return df_unificado

# Aplicar la función
business_normal = unificar_registros(business_normal)
business_normal.drop(columns=['business_id'], inplace=True)

In [12]:
business_normal.to_csv(r'data_normalizada\business_normal.csv', index = False)
reviews.to_csv(r'data_normalizada\reviews_normal.csv', index = False)
reviews.to_parquet(r'data_normalizada\reviews_normal.parquet', index = False)